In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# from autocorrect import spell
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/Combined_News_DJIA.csv")
df.head()

In [ ]:
def tokenize_text(text):
    stop = set(stopwords.words('english'))
    text = text.lower()
    # tokenizer = get_tokenizer("en_US")
    tokenizer = RegexpTokenizer(r'[a-z]+')  # strips non-alphabetical characters
    # cleaned_tokens = text_col.apply(lambda row: spell(tokenizer.tokenize(row)).lower)
    cleaned_tokens = [spell(word) for word in tokenizer.tokenize(text) if word not in stop]
    # tagged_tokens = nltk.pos_tag(cleaned_tokens)
    return cleaned_tokens

def clean_and_tag(text, tokenizer):
    stop = set(stopwords.words('english'))
    text = str(text).lower()  # apply(lambda x: x.lower())
    # tokenizer = get_tokenizer("en_US")
    cleaned_tokens = [word for word in tokenizer.tokenize(text) if word not in stop and len(word) > 1]
    tagged_tokens = nltk.pos_tag(cleaned_tokens)
    return tagged_tokens

def lemmatize_tokens(tokens_col, lemma):
    tokens_col = tokens_col.apply(lambda row: [get_wordnet_pos(word) for word in row if get_wordnet_pos(word)[1] != ''])
    lemmatized_words = tokens_col.apply(lambda row: [lemma.lemmatize(word[0],word[1]) for word in row])  # [lemma.lemmatize(x) for x in noun_tokens]
    return lemmatized_words

# Convert to wordnet part of speech from UPenn
def get_wordnet_pos(word_tag_tuple):
    tag = word_tag_tuple[1]
    if tag.startswith('J'):  # adjectives
        tag = 'a'
    elif tag.startswith('V'):  # verbs
        tag = 'v'
    elif tag.startswith('N'):  # nouns
        tag = 'n'
    elif tag.startswith('R'):  # adverbs
        tag = 'r'
    else:                      # default tag in nltk lemmatizer is noun
        tag = ''
    return (word_tag_tuple[0], tag)

In [ ]:
tokenizer = RegexpTokenizer(r'[a-z]+')  # strips non-alphabetical characters
lemma = WordNetLemmatizer()

end = 3
headline_cols = ['0']
for x in range(1, end):
    headline = df['Top%s' %x] 
    tokenized_headline = headline.apply(lambda row: clean_and_tag(row, tokenizer))
    print(tokenized_headline)
    lemmatized_headline = lemmatize_tokens(tokenized_headline, lemma)
    print(lemmatized_headline)
    # temp = temp.concat(headline_col, axis=1)
    headline_cols.append(lemmatized_headline)
temp = pd.concat(headline_cols[1:end-1])
temp.head()
temp.columns

In [ ]:

for x in range(1, end):